In [1]:
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

%matplotlib inline

In [2]:
datain = np.loadtxt('13a_input.txt',comments=None,delimiter='\n',dtype=str)#,usecols=(0,2))#,delimiter=' ')
#datain = np.loadtxt('13a_input_test.txt',comments=None,delimiter='\n',dtype=str)#,usecols=(0,2))#,delimiter=' ')

In [3]:
#datain

In [4]:
arrdatain = np.zeros((len(datain[0]),len(datain))).astype(str)

#convert datain to np array
for i in range(arrdatain.shape[0]):
    for j in range(arrdatain.shape[1]):
        arrdatain[i,j] = datain[j][i]
        
#arrdatain.T

In [5]:
# Find carts, note their position and direction, replace them with tracks
cars = []

for i in range(arrdatain.shape[0]):
    for j in range(arrdatain.shape[1]):
        if arrdatain[i,j] == '>':
            cars.append([i,j,'r'])
            arrdatain[i,j] = '-'
            
        if arrdatain[i,j] == 'v':
            cars.append([i,j,'d'])
            arrdatain[i,j] = '|' 
            
        if arrdatain[i,j] == '<':
            cars.append([i,j,'l'])
            arrdatain[i,j] = '-' 
            
        if arrdatain[i,j] == '^':
            cars.append([i,j,'u'])
            arrdatain[i,j] = '|'
            
cars
#print(arrdatain.T)

[[6, 34, 'd'],
 [9, 15, 'r'],
 [12, 86, 'd'],
 [47, 22, 'u'],
 [47, 129, 'l'],
 [50, 146, 'l'],
 [61, 4, 'l'],
 [69, 95, 'l'],
 [74, 26, 'r'],
 [76, 15, 'l'],
 [78, 128, 'r'],
 [107, 53, 'd'],
 [112, 50, 'd'],
 [125, 108, 'd'],
 [133, 102, 'r'],
 [136, 56, 'd'],
 [137, 100, 'd']]

In [6]:
class Cars:
    def __init__(self,xy):
        self.xy = np.array(xy) # location of car
        self.udlr = None # direction of travel: 'u','d','l','r'
        self.nxturn = 'L' # at intersections, the cars always turn L=>S=>R=>L=>... etc.
        
    def move(self):
        if self.udlr == 'u':
            self.xy += np.array([0,-1])
            if arrdatain[self.xy[0],self.xy[1]] == '/':
                self.cw()
            elif arrdatain[self.xy[0],self.xy[1]] == '\\':
                self.ccw()
            elif arrdatain[self.xy[0],self.xy[1]] == '+':
                self.turn()
            
        if self.udlr == 'd':
            self.xy += np.array([0,1])
            if arrdatain[self.xy[0],self.xy[1]] == '/':
                self.cw()
            elif arrdatain[self.xy[0],self.xy[1]] == '\\':
                self.ccw()
            elif arrdatain[self.xy[0],self.xy[1]] == '+':
                self.turn()
            
        if self.udlr == 'l':
            self.xy += np.array([-1,0])
            if arrdatain[self.xy[0],self.xy[1]] == '/':
                self.ccw()
            elif arrdatain[self.xy[0],self.xy[1]] == '\\':
                self.cw()
            elif arrdatain[self.xy[0],self.xy[1]] == '+':
                self.turn()
            
        if self.udlr == 'r':
            self.xy += np.array([1,0])
            if arrdatain[self.xy[0],self.xy[1]] == '/':
                self.ccw()
            elif arrdatain[self.xy[0],self.xy[1]] == '\\':
                self.cw()
            elif arrdatain[self.xy[0],self.xy[1]] == '+':
                self.turn()

            
    
    def cw(self):
        self.udlr = ['u','r','d','l','u'][['u','r','d','l','u'].index(self.udlr)+1]
        
    def ccw(self):
        self.udlr = ['u','l','d','r','u'][['u','l','d','r','u'].index(self.udlr)+1]
        
    def turn(self):
        if self.nxturn == 'L':
            self.ccw()
        elif self.nxturn == 'R':
            self.cw()
        
        self.nxturn = ['L','S','R','L'][['L','S','R','L'].index(self.nxturn)+1]


def coll_search(car_dictionary):
    # Test for collision using dictionary sets returns True when collision occurrs 
    list_xy = [] # initialize list
    for key,item in car_dictionary.items():
        list_xy.append(str(item.xy.tolist())) # populate list with strings
        
    return len(set(list_xy)) != len(list_xy)

def coll_loc (car_dictionary):
    # find the location of the collision 
    list_xy = [] # initialize list
    for key,item in car_dictionary.items():
        list_xy.append(str(item.xy.tolist())) # populate list with strings
        
    seen = {}
    dupes = []

    for item in list_xy:
        if item not in seen:
            seen[item] = 1
        else:
            if seen[item] == 1:
                dupes.append(item)
            seen[item] += 1
    
    return dupes


def findorder(car_dictionary):
    # Sort cars by coordinate on y coordinate, then x coordinate
    dtype = [('car', int), ('x', int), ('y', int)]
    arr_car = np.zeros((len(car_dictionary))).astype(dtype)
    for i in car_dictionary:
        arr_car[i] = i,car_dictionary[i].xy[0],car_dictionary[i].xy[1]

    # Then sort
    arr_car = np.sort(arr_car, order=['y', 'x'])

    # Then extract the order
    order = []
    for i in range(len(cdic)):
        order.append(arr_car[i][0])
        
    return order

In [7]:
# Create a Car object for each car and a dictionary to find them

cdic = {}

for i in range(len(cars)):
    vars()['car'+str(i)] = Cars(i)
    cdic[i] = vars()['car'+str(i)]
    vars()['car'+str(i)].xy = np.array([cars[i][0],cars[i][1]])
    vars()['car'+str(i)].udlr = cars[i][2]
    
 

In [8]:
# Run this:

run = True
tick = 0

while run == True:
    
    order = findorder(cdic)

    for car in order:
        cdic[car].move()
        if coll_search(cdic) == True:
            print('Collision Detected at:')
            print(coll_loc(cdic))
            run = False
            break
            
    tick += 1
#    print('Tick',tick)
#    print(cdic[0].xy,cdic[1].xy)
        
# not 31,146    
    
    


Collision Detected at:
['[31, 146]']
